In [ ]:
# Install ultralytics (YOLO)
%pip install ultralytics -q

# Verify installation
import ultralytics
print(f"✓ Ultralytics version: {ultralytics.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Ultralytics version: 8.4.12


In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path
import shutil
from google.colab import files
print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [ ]:
# Plant names mapping (class_00 to class_32)
PLANT_NAMES = [
    'Aerva lanata (Bui Jhar)',
    'Aloe vera (Ghiu Kumari)',
    'Andrographis paniculata (Kalmegh)',
    'Bacopa monnieri (Brahmi)',
    'Calotropis gigantea (Aank)',
    'Centella asiatica (Ghodtapre)',
    'Chromolaena odorata (Banmara)',
    'Cissus quadrangularis (Hadjod)',
    'Clerodendrum infortunatum (Bharangi)',
    'Clitoria ternatea (Aparajita)',
    'Coccinia grandis (Kundru)',
    'Curcuma longa (Besar)',
    'Datura metal linn (Dhatura)',
    'Desmodium gangeticum (Salparni)',
    'Hemigraphis colorata (Muriko Jhar)',
    'Kaempferia galanga linn (Kachur)',
    'Mentha spicata (Pudina)',
    'Ocimum gratissimum (Ban Tulsi)',
    'Ocimum tenuiflorum (Tulsi)',
    'Oxalis corniculata (Chari Amilo)',
    'Peperomia pellucida (Jalpapdi)',
    'Phyllanthus niruri (Bhui Amala)',
    'Piper betle (Paan)',
    'Piper longum (Pipla)',
    'Piper nigrum (Marich)',
    'Plectranthus barbatus (Patharchur)',
    'Plumbago indica (Chitrak)',
    'Pseudarthria viscida (Saliparni)',
    'Ricinus communis linn (Aandi)',
    'Tinospora cordifolia (Gurjo)',
    'Neem (Azadirachta indica)',
    'Guava (Psidium guajava)',
     'Mango (Mangifera indica)'
]

NUM_CLASSES = len(PLANT_NAMES)
print(f"✓ Loaded {NUM_CLASSES} plant species")
print("\nPlant Species:")
for i, name in enumerate(PLANT_NAMES):
    print(f"  class_{i:02d}: {name}")

✓ Loaded 33 plant species

Plant Species:
  class_00: Aerva lanata (Bui Jhar)
  class_01: Aloe vera (Ghiu Kumari)
  class_02: Andrographis paniculata (Kalmegh)
  class_03: Bacopa monnieri (Brahmi)
  class_04: Calotropis gigantea (Aank)
  class_05: Centella asiatica (Ghodtapre)
  class_06: Chromolaena odorata (Banmara)
  class_07: Cissus quadrangularis (Hadjod)
  class_08: Clerodendrum infortunatum (Bharangi)
  class_09: Clitoria ternatea (Aparajita)
  class_10: Coccinia grandis (Kundru)
  class_11: Curcuma longa (Besar)
  class_12: Datura metal linn (Dhatura)
  class_13: Desmodium gangeticum (Salparni)
  class_14: Hemigraphis colorata (Muriko Jhar)
  class_15: Kaempferia galanga linn (Kachur)
  class_16: Mentha spicata (Pudina)
  class_17: Ocimum gratissimum (Ban Tulsi)
  class_18: Ocimum tenuiflorum (Tulsi)
  class_19: Oxalis corniculata (Chari Amilo)
  class_20: Peperomia pellucida (Jalpapdi)
  class_21: Phyllanthus niruri (Bhui Amala)
  class_22: Piper betle (Paan)
  class_23: Piper

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

dataset_path="/content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent"


IMAGE_SIZE = 224

MODEL_NAME = 'yolov8n-cls.pt'  # Options: yolov8n-cls.pt, yolov8s-cls.pt, yolov8m-cls.pt, yolov8l-cls.pt, yolov8x-cls.pt
EPOCHS = 100
BATCH_SIZE = 16  # Adjust based on GPU memory (16 for T4, 32+ for A100)
DEVICE = 0  # 0 for GPU, 'cpu' for CPU
WORKERS = 8
OPTIMIZER = 'AdamW'
LEARNING_RATE = 0.01

PROJECT_NAME = '/content/drive/MyDrive/YOLOClassification32classes2ndTry/runs/classify'
EXPERIMENT_NAME = 'medicinal_plants'


print("TRAINING CONFIGURATION")

print(f"Dataset: {dataset_path}")
print(f"Model: {MODEL_NAME}")
print(f"Image Size: {IMAGE_SIZE}x{IMAGE_SIZE} ⚠️ REMEMBER THIS FOR PREDICTION!")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Device: {DEVICE}")
print(f"Optimizer: {OPTIMIZER}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Project Name: {PROJECT_NAME}") # Added for verification
print("="*60)
print(f"\n⚠️ IMPORTANT: Write down IMAGE_SIZE = {IMAGE_SIZE} for prediction notebook!")

TRAINING CONFIGURATION
Dataset: /content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent
Model: yolov8n-cls.pt
Image Size: 224x224 ⚠️ REMEMBER THIS FOR PREDICTION!
Epochs: 100
Batch Size: 16
Device: 0
Optimizer: AdamW
Learning Rate: 0.01
Project Name: /content/drive/MyDrive/YOLOClassification32classes2ndTry/runs/classify

⚠️ IMPORTANT: Write down IMAGE_SIZE = 224 for prediction notebook!


In [ ]:
# Initialize YOLO classification model
model = YOLO(MODEL_NAME)

print(f"✓ Model initialized: {MODEL_NAME}")
print(f"✓ Ready to train on {NUM_CLASSES} plant species")

✓ Model initialized: yolov8n-cls.pt
✓ Ready to train on 33 plant species


In [ ]:
# Train the model
print("🚀 Starting training...")
print(f"Training on {NUM_CLASSES} medicinal plant species")
print("This may take a while depending on dataset size and epochs...\n")

results = model.train(
    data=dataset_path,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,  # ⚠️ CRITICAL: Remember this value!
    batch=BATCH_SIZE,
    device=DEVICE,
    workers=WORKERS,
    project=PROJECT_NAME,
    name=EXPERIMENT_NAME,
    exist_ok=True,
    pretrained=True,
    optimizer=OPTIMIZER,
    lr0=LEARNING_RATE,
    verbose=True,
    seed=42,
    deterministic=True,
    single_cls=False,
    val=True
)

print("\n" + "="*60)
print("✓ TRAINING COMPLETED!")
print("="*60)

🚀 Starting training...
Training on 33 medicinal plant species
This may take a while depending on dataset size and epochs...

Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cl

In [ ]:
# Display training results
print("Training Results:")
print(results)

# The best model will be saved at: runs/classify/medicinal_plants/weights/best.pt
# Corrected path construction based on actual save_dir from results
best_model_path = results.save_dir / 'weights' / 'best.pt'

if os.path.exists(best_model_path):
    print(f"\n✓ Best model saved at: {best_model_path}")
    print(f"✓ Model size: {os.path.getsize(best_model_path) / (1024*1024):.2f} MB")
else:
    print("\n⚠ Best model path not found. (This message should not appear after correction)")

Training Results:
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e073f156a20>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: PosixPath('/content/drive/MyDrive/YOLOClassification32classes2ndTry/runs/classify/medicinal_plants')
speed: {'preprocess': 0.20110145411736619, 'inference': 0.860518950588919, 'loss': 0.010798680003277054, 'postprocess': 0.03406550822974703}
task: 'classify'
top1: 1.0
top5: 1.0

✓ Best model saved at: /content/drive/MyDrive/YOLOClassification32classes2ndTry/runs/classify/medicinal_plants/weights/best.pt
✓ Model size: 2.91 MB


In [ ]:
# Load the best model
best_model = YOLO(best_model_path)

# Validate on test set
print("Validating on test set...")
metrics = best_model.val(data=dataset_path)

print("\n" + "="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"Top-1 Accuracy: {metrics.top1:.4f} ({metrics.top1*100:.2f}%)")
print(f"Top-5 Accuracy: {metrics.top5:.4f} ({metrics.top5*100:.2f}%)")
print("="*60)

Validating on test set...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,477,153 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent/train... found 5539 images in 33 classes ✅ 
val: /content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent/valid... found 425 images in 33 classes ✅ 
test: /content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent/test... found 425 images in 33 classes ✅ 
val: Fast image access ✅ (ping: 0.5±0.3 ms, read: 3.5±2.0 MB/s, size: 5.8 KB)
val: Scanning /content/drive/MyDrive/YOLOClassificatio/dataset_subsetyolo32class10percent/valid... 425 images, 0 corrupt: 100% ━━━━━━━━━━━━ 425/425 148.5Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 27/27 8.6it/s 3.2s
                   all          1          1
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.0ms po